In [7]:
#Dependencies
from bs4 import BeautifulSoup
import requests
import json
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile,printSoupToHtml
from Product import ProductDetailDTO

async def scrape_extra_full(url):
    #change request from en-sa to ar-sa
    if url.find("/en-sa/")!=-1:
        url=url.replace("/en-sa/","/ar-sa/")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    #printSoupToHtml(soup.prettify(),'extra.html')

    # Find the <script> tag containing ACC.config.productDetails
    script_tag = soup.find('script', {'type': 'text/javascript'})

    if script_tag:
        # Extract the JavaScript content from the <script> tag
        script_content = script_tag.string
        
        # Locate and extract the ACC.config.productDetails object
        start_index = script_content.find('ACC.config.productDetails')
        stop_index = script_content.find('ACC.config.currentCity')

        
        if start_index != -1 and stop_index != -1:
            product_details = script_content[start_index:stop_index]
            product_details = product_details.replace('ACC.config.productDetails = ', '').strip()
            product_details = product_details[:-1]
            product_details = json.loads(product_details)
            #printJsonToFile(product_details,'extra.json')
            product_details = json.loads(open('extra.json').read())
            name_global = product_details.get('nameEn', 'N/A')
            name_local = product_details.get('displayName', 'N/A')
            isAvailable = product_details.get('stock', {}).get('stockLevel', 'N/A')>0
            total_price = product_details.get('price', {}).get('formattedValue', 'N/A')
            discount = product_details.get('percentageDiscount', {}).get('value', 'N/A')
            PriceAfterDiscount = float(total_price) - (float(total_price) * discount / 100)
            description = product_details.get('descriptionEn', 'N/A')
            description_local = product_details.get('summary', 'N/A')
            rating = product_details.get('rating', 'N/A')
            image= product_details.get('highlights')
            soup = BeautifulSoup(image, 'html.parser')
            p=url.split("/p/")[1].split("/")[0]
            images = [f"https://media.extra.com/i/aurora/{p}_100_0{i}" for i in range(1,6)]
            result = []
            for classification in product_details.get("classifications", []):
                for feature in classification.get("features", []):
                    name = feature.get("name")
                    value = feature.get("featureValues", [{}])[0].get("value")
                    if name and value:
                        result.append(f"{name}:{value}")
            item_data = ProductDetailDTO(name_Global=name_global,name_Local=name_local,is_available=isAvailable,description_Local=result, price=PriceAfterDiscount, rating=rating, description_Global=description,images=images,productlink1=url)
            return item_data
#x=await scrape_extra_full('https://www.extra.com/ar-sa/mobiles-tablets/mobiles/smartphone/samsung-galaxy-z-fold-5-5g-512gb-icy-blue/p/100343500')   
url="https://www.extra.com/en-sa/mobiles-tablets/mobiles/smartphone/apple-iphone-15-plus-5g-6-7-inch-128gb-black/p/100345666"
x=await scrape_extra_full(url)
print("name_global:",x.name_Global)
print("name_local:",x.name_Local)
print("isAvailable:",x.is_available)    
print("price:",x.price)
print("rating:",x.rating)
print("description:",x.description_Global)
print("Description_Local:",x.description_Local)
print("images:",x.images)




name_global: Apple iPhone 15 Plus, 5G, 6.7 inch, 128GB, Black
name_local: ÃÈá ÃíÝæä 15 ÈáÓ¡ 5Ìí¡ 6.7 ÈæÕÉ¡ 128 ÌíÌÇ¡ ÃÓæÏ
isAvailable: True
price: 3699.319
rating: 4.4
description: MU083AH/A--Apple iPhone 15 Plus, 5G, 6.7 inch, Super Retina XDR Display, OLED,A16 Bionic chip, 128GB, Black, IOS 17, 12MP Ultra Wide +  48MP Wide  Rear Camera, 12 TrueDepth Front Camera, FACE ID
Description_Local: ['äÙÇã ÇáÊÔÛíá:Âí Çæ ÃÓ 17', 'ãÏÎá ÈØÇÞÉ ÇáÐÇßÑÉ:ÛíÑ ãÊØÇÈÞ', 'ãÓÇÍÉ ÇáÐÇßÑÉ ÇáÚÔæÇÆíÉ (ÑÇã):ÛíÑ ãÊØÇÈÞ', 'Çááæä:ÃÓæÏ', 'ÔÈßÉ ÇáÇÊÕÇá:ÇáÌíá ÇáÎÇãÓ', 'ÍÌã ÇáÔÇÔÉ(ÈÇáÈæÕÉ):6.7', 'ßÇãíÑÇ ÎáÝíÉ:48MP + 12MP', 'ÐÇßÑÉ / ÏÇÎáíÉ:128ÌíÌÇÈíÊ', 'ÇáØæá:ÛíÑ ãÊØÇÈÞ', 'ÇáæÒä:Not Applicable', 'ÕäÚ Ýì:ãÊäæÚ', 'äæÇÉ ÇáãÚÇáÌ:Åíå 16 Èíæäß', 'ÍÌã ÇáÈØÇÑíÉ (ãááí ÃãÈíÑ):ÛíÑ ãÊØÇÈÞ', 'ÏÞÉ ÇáßÇãíÑÇ ÇáÃãÇãíÉ:12ãíÌÇÈßÓá', 'ãÞÇæã ááãÇÁ:äÚã', 'ãÇÓÍ ÇáÈÕãÉ(äÚã/áÇ):áÇ', 'ÇÓã ÇáÓáÓáÉ:ÂíÝæä 15', 'ÇáÚÑÖ:ÛíÑ ãÊØÇÈÞ', 'ÇáÇÑÊÝÇÚ:ÛíÑ ãÊØÇÈÞ', 'ÇáÑÄíÇ ãä ÎáÇá ÇáÛØÇÁ:ÛíÑ ãÊØÇÈÞ', 'ÓÑÚÉ ÇáãÚÇáÌ:ÛíÑ ãÊØÇÈÞ', 'äæÚ ÇáÔÇÔÉ:ÓæÈÑ ÑíÊäÇ ÅßÓ Ïí ÂÑ